##### First we import all libraries required for this notebook

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests
from bs4 import BeautifulSoup

##### Read the Toronto postal code data from wiki
##### Use the BeautifulSoup library to scrap the data

In [2]:
toronto_neigh  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
toronto_neigh_text = toronto_neigh.text
toronto_neigh_soup = BeautifulSoup(toronto_neigh_text, "lxml")

##### Create dataframe with 3 columns using pandas. 

In [3]:
pd.options.display.max_rows = 999
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
# instantiate the dataframe
toronto_nhoods = pd.DataFrame(columns=column_names)

##### Scrap the data by reading the main table using its classname. 
##### Read all rows. If the row does not contain 3  td's then ignore this row as this row will not contain relevant data
##### Check if value of Borough is 'Not assigned'. If it is, then ignore this row.
##### If the value of Borough is 'Not assigned' but the value of Neighborhood is 'Not assigned', then assign value of Burrough to Neighborhood for this row.
##### FILL the dataframe with data for 3 columns

In [4]:
table_main = toronto_neigh_soup.find('table', class_='wikitable sortable')

tablebody_main = table_main.find('tbody')
#print(tablebody_main)
tablerows_all = tablebody_main.find_all('tr')

for row in tablerows_all:
    td_postalcode = row.find_all('td')
    
    if len(td_postalcode) == 3:
        if "Not assigned" not in td_postalcode[1]:
            
            if (td_postalcode[2].text.replace("\n","") == "Not assigned"):
                nName = td_postalcode[1].text
            else:
                nName = td_postalcode[2].text.replace("\n","")
                                
            toronto_nhoods = toronto_nhoods.append({'PostalCode': td_postalcode[0].text ,
                                                  'Borough': td_postalcode[1].text ,
                                                  'Neighborhood': nName,
                                                      }, ignore_index=True)
    
    
    

##### Verify the data

In [5]:
toronto_nhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


##### We want to group the data by Postalcode, Borough and then concatenate (join) Neighborhood column so that we get a unique row for each Postalcode/Borough 

In [6]:
toronto_nhood_grouped = toronto_nhoods.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
toronto_nhood_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### Use the shape method to get the number of rows and columns in the dataframe
##### There are 103 rows and 3 columns

In [7]:
toronto_nhood_grouped.shape

(103, 3)

In [8]:
#!conda install -c conda-forge geocoder --yes
#import geocoder
#toronto_nhoods_LatLong = toronto_nhood_grouped
#HeaderList_LatLong = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', "Longitude"]
#toronto_nhoods_LatLong = toronto_nhoods_LatLong.reindex(columns = HeaderList_LatLong)  

#pc = 'M1E'
#g = geocoder.google('{}, Toronto, Ontario'.format(pc))
#lat_lng_coords = g.latlng

#for index, row in toronto_nhoods_LatLong.iterrows():
#    pc = row["PostalCode"]
#    lat_lng_coords = None
    
    #while(lat_lng_coords is None):
#    for i in range(1,20):
#        if (lat_lng_coords is None):
#            print(pc + '   '  + i + '    A')
#            g = geocoder.google('{}, Toronto, Ontario'.format(pc))
#            lat_lng_coords = g.latlng
#        else:
#            print(pc + '   '  + i + '   ' + lat_lng_coords)


#<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
#None

In [9]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_data_latlong.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [11]:
df_data_latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
toronto_nhood_full = pd.merge(toronto_nhood_grouped, df_data_latlong, on="PostalCode")

In [13]:
toronto_nhood_full.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
